In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 17.012 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 1.38 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 1.737 seconds


In [3]:
%%hive
DROP TABLE IF EXISTS conteo;
CREATE TABLE conteo
AS
SELECT YEAR(c4) AS year, c5 AS palabras FROM tbl0;

DROP TABLE IF EXISTS conteo;
OK
Time taken: 0.097 seconds
CREATE TABLE conteo
AS
SELECT YEAR(c4) AS year, c5 AS palabras FROM tbl0;
Query ID = root_20200130224220_a2f3df86-c22a-4ae2-88b1-7691094f74b3
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Job running in-process (local Hadoop)
2020-01-30 22:42:24,970 Stage-1 map = 100%,  reduce = 0%
Ended Job = job_local1524832669_0001
Stage-4 is selected by condition resolver.
Stage-3 is filtered out by condition resolver.
Stage-5 is filtered out by condition resolver.
Moving data to directory file:/user/hive/warehouse/.hive-staging_hive_2020-01-30_22-42-20_165_4704979527108785688-1/-ext-10002
Moving data to directory file:/user/hive/warehouse/conteo
MapReduce Jobs Launched: 
Stage-Stage-1:  HDFS Read: 0 HDFS Write: 0 SUCCESS
Total MapReduce CPU Time Spent: 0 msec
OK
Time taken: 5.994 seconds


In [4]:
%%hive
INSERT OVERWRITE LOCAL DIRECTORY 'output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
STORED AS TEXTFILE

SELECT year, letras, count(1)
FROM conteo LATERAL VIEW explode(palabras) adTable AS letras
GROUP BY year, letras;

INSERT OVERWRITE LOCAL DIRECTORY 'output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
STORED AS TEXTFILE
SELECT year, letras, count(1)
FROM conteo LATERAL VIEW explode(palabras) adTable AS letras
GROUP BY year, letras;
Query ID = root_20200130224234_ad19b049-ed83-4dd3-a74c-3a7c5709eb24
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Job running in-process (local Hadoop)
2020-01-30 22:42:35,825 Stage-1 map = 100%,  reduce = 100%
Ended Job = job_local896536255_0002
Moving data to local directory output
MapReduce Jobs Launched: 
Stage-Stage-1:  HDFS Read: 0 HDFS Write: 0 SUCCESS
Total MapReduce CPU Time Spent: 0 msec
OK
Time taken: 1.93 second

In [5]:
%hive_quit